# Classification of power spectra with BaCoN

Arnav Agarwal

Updated training notebook including minimum k_mode parameters, padding parameter, compatibility with TPU execution, github push/pull compatibility and minor improvements.

If you are saving models, you will need to use git push within the colab environment, or download the model files

## Load the code

Here we import the github repository and import BaCoN.

Tensorflow2 and Tensorflow2 Probability should be installed.

In [6]:
!git clone https://github.com/arnava13/SeniorHonoursProject
%cd SeniorHonoursProject/BaCoN-II

Cloning into 'SeniorHonoursProject'...
remote: Enumerating objects: 228625, done.
remote: Counting objects: 100% (15415/15415), done.
remote: Compressing objects: 100% (12968/12968), done.
remote: Total 228625 (delta 2491), reused 15321 (delta 2439), pack-reused 213210
Receiving objects: 100% (228625/228625), 2.72 GiB | 20.55 MiB/s, done.
Resolving deltas: 100% (2622/2622), done.
Updating files: 100% (75854/75854), done.
/content/SeniorHonoursProject/BaCoN-II


## Run training in shell
only do this when not loading trained network

In [7]:
!pip install -q lib5c
!lib5c -v
!pip install tensorflow==2.15.0
!pip install tensorflow-probability==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
lib5c version 0.6.1


In [8]:
import os;
import subprocess
import threading
import argparse
from test import *
from utils import *
from models import *
from data_generator import *
from train import *

In [9]:
from importer import load_model_for_test, my_predict, predict_bayes_label, predict_mean_proba

In [10]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'serif'
plt.rcParams["mathtext.fontset"] = "cm"

In [11]:
def handle_output(stream, log_file):
    for line in iter(stream.readline, b''):  # This iterates over the output line by line
        my_bytes = line.strip()
        if my_bytes:
            output = f"{my_bytes.decode('utf-8')}"
            print(output)
            log_file.write(output + "\n")

### set parameters for training

In [12]:
# Training options:

# ------------------ change the training directory and model name here ------------------

# training data directory path
DIR='data/fivelabel_train'
## (batch size, is a multiple of #classes * noise realisations, e.g. 5 classes, 10 noise --> must be multiple of 50)
batch_size='6250'

# training the network
GPU='True' # Use GPUs in training?
TPU='False' # Train with TPUs?
# which kind of model
bayesian='True' # Bayesian NN or traditional NN (weights single valued or distributions?)
n_epochs='50' # How many epochs to train for?


# ------------------ other parameters ------------------

# Directories
mypath=None # Parent directory

# Edit this with base model directory
mdir='models/'

# normalisation file
norm_data_name = '/planck_ee2.txt'

# scale cut and resolution
k_max='2.5' # max of k-modes to include?
k_min='0.0' # min of k-modes to include?
sample_pace='2'

# should the processed curves be saved in an extra file (False recommended, only efficient for n_epochs = 1)
save_processed_spectra='False'

# fine-tune = only 'LCDM' and 'non-LCDM'
fine_tune='False' # Model trained to distinguish between 'LCDM' and 'non-LCDM' classes or between all clases. i.e. binary or multi-classifier

# Cache directory, set as False to cache on memory.
cache_dir='False'

# -------------------- noise model --------------------

# noise model - Cosmic variance and shot noise are Euclid-like
n_noisy_samples='10' # How many noise examples to train on?
add_noise='True'# Add noise?
add_shot='False'# Add shot noise term? -- default should be False
add_sys='True'# Add systematic error term?
add_cosvar='True'# Add cosmic variance term?

# path to folder with theory error curves
curves_folder = 'data/curve_files_sys/theory_error'; sigma_curves_default = '0.05'
# change rescale distribution of theory error curves, uniform recommended
rescale_curves = 'uniform' # or gaussian or None

# sys error curves - relative scale of curve-amplitude
sigma_curves='0.05'

# ----------------- Model Name Generation -------------

#Two or Six label?
"""
if fine_tune != 'True':
  two_or_six = 'sixlabel'
else:
  two_or_six = 'twolabel'
"""

two_or_six = 'fivelabel'

#Examples Per Class?
examples_per_class = 10000

#Generation code?
gen_code = "EE2"

#LCDM * Filter or Equal Examples from each class * Filter for randoms?
#randstype = "equalexamples-randoms"
randstype = "LCDM-randoms"

#Additional Notes?
fname_notes = ""

#Assemble fname
fname = f"{two_or_six}_{examples_per_class}_{gen_code}_{randstype}_kmin-{k_min}_kmax-{k_max}_{fname_notes}"

# -------------------- additional training settings --------------------


# Type of model
model_name='custom' # Custom or dummy - dummy has presets as given in models.py

#Test mode?
test_mode='False' # Network trained on 1 batch of minimal size or not?
seed='1312' # Initial seed for test mode batch
# Saving or restoring training
restore='False' # Restore training from a checkpoint?
save_ckpt='True' # Save checkpoints?


val_size='0.15' # Validation set % of data set

add_FT_dense='False' #if True, adds an additional dense layer before the final 2D one

patience='20' # terminate training after 'patience' epochs if no decrease in loss function
lr='0.01' # learning rate
decay='0.95' #decay rate: If None : Adam(lr),

padding='valid'

# -------------------- BNN parameters -----------------

# Example image details
im_depth='500' # Number in z direction (e.g. 500 wave modes for P(k) examples)
im_width='1' # Number in y direction (1 is a 2D image : P(k,mu))
im_channels='4'  # Number in x direction (e.g. 4 redshifts for P(k,z))
swap_axes='True' # Do we swap depth and width axes? True if we have 1D image in 4 channels
sort_labels='True' # Sorts labels in ascending/alphabetical order

z1='0' # which z-bins to include? Assumes each channel is a new z bin.
z2='1' # which z-bins to include? Assumes each channel is a new z bin.
z3='2' # which z-bins to include? Assumes each channel is a new z bin.
z4='3' # which z-bins to include? Assumes each channel is a new z bin.

# Number of layers and kernel sizes
k1='10'
k2='5'
k3='2'
 # The dimensionality of the output space (i.e. the number of filters in the convolution)
f1='8'
f2='16'
f3='32'
 # Stride of each layer's kernel
s1='2'
s2='2'
s3='1'
# Pooling layer sizes
p1='2'
p2='2'
p3='0'
# Strides in Pooling layer
sp1='2'
sp2='1'
sp3='0'

n_dense='1' # Number of dense layers

# labels of different cosmologies
#c0_label = 'lcdm'
#c1_label = 'fR dgp wcdm'

log_path = mdir + fname + '_log'

if fine_tune == "True":
    log_path += '_FT'

log_path += '.txt'




### run the training
this may take a while to run, also expect some (harmless) error messages in the output

In [13]:
# -------------- training loads parameters entered above  --------------
proc = subprocess.Popen(["python3", "train.py", "--test_mode" , test_mode, "--seed", seed, \
                     "--bayesian", bayesian, "--model_name", model_name, \
                     "--fine_tune", fine_tune, "--log_path", log_path,\
                     "--restore", restore, \
                     "--models_dir", mdir, \
                     "--cache_dir", cache_dir, \
                     "--fname", fname, \
                     "--DIR", DIR, \
                     '--norm_data_name', norm_data_name, \
                     '--curves_folder', curves_folder,\
                     "--c_0", 'lcdm', \
                     "--c_1", 'fR', 'dgp', 'wcdm', 'ds', 'rand'\
                     "--save_ckpt", save_ckpt, \
                     "--im_depth", im_depth, "--im_width", im_width, "--im_channels", im_channels, \
                     "--swap_axes", swap_axes, \
                     "--sort_labels", sort_labels, \
                     "--add_noise", add_noise, "--add_shot", add_shot, "--add_sys", add_sys,"--add_cosvar", add_cosvar, \
                     "--sigma_curves", sigma_curves, \
                     "--sigma_curves_default", sigma_curves_default, \
                     "--save_processed_spectra", save_processed_spectra, \
                     "--sample_pace", sample_pace,\
                     "--n_noisy_samples", n_noisy_samples, \
                     "--rescale_curves", rescale_curves, \
                     "--val_size", val_size, \
                     "--z_bins", z1,z2,z3,z4, \
                     "--filters", f1,f2,f3, "--kernel_sizes", k1,k2,k3, "--strides", s1,s2,s3, "--pool_sizes", p1,p2,p3, "--strides_pooling", sp1,sp2,sp3, \
                     "--k_max", k_max,\
                     "--k_min", k_min,\
                     "--n_dense", n_dense,\
                     "--add_FT_dense", add_FT_dense, \
                     "--n_epochs", n_epochs, "--patience", patience, "--batch_size", batch_size, "--lr", lr, \
                     "--decay", decay, \
                     "--GPU", GPU, \
                     "--TPU", TPU, \
                     "--padding", padding],\
                     stdout=subprocess.PIPE, \
                     stderr=subprocess.PIPE)

with open(log_path, "w") as log_file:

    stdout_thread = threading.Thread(target=handle_output, args=(proc.stdout, log_file))
    stderr_thread = threading.Thread(target=handle_output, args=(proc.stderr, log_file))

    stdout_thread.start()
    stderr_thread.start()

    stdout_thread.join()
    stderr_thread.join()

    proc.wait()

2024-03-29 01:23:26.560607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 01:23:26.560657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 01:23:26.561834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 01:23:27.601853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Creating directory models/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5_
Logger creating log file: models/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5_/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5__log.txt
-------- Parameter

In [ ]:
ls models/

## Run the test of the model
- insert the path+name of the log file of the model
- change the test directory
- change the custom name that will be used for the confusion matrix

The pdf file with the confusion matrix is saved in the model folder.

In [23]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pylatex

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
pandoc set to manually installed.
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 libapache-pom-java libcommons-logging-java libcommons-parent-java
  libfontbox-java libfontenc1 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libkpathsea6
  libpdfbox-java libptexenc1 libruby3.0 libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1
  libzzip-0-13 lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet ruby-rubygems
  ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils teckit tex-common tex-gyre
  texlive-base texlive-binaries texlive-fonts-recommended texlive-latex-base
  texlive-latex-recommended texlive-pictures texlive-plain-generic tipa xfonts-encodings
  xfonts-utils
Suggested packages:
  fo

In [25]:
!python test.py --log_path='models/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5_/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5__log.txt' \
  --TEST_DIR='data/fivelabel_test' --cm_name_custom='fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5_'

2024-03-29 03:43:43.354096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 03:43:43.354150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 03:43:43.355940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 03:43:44.376477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Reading log from models/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5_/fivelabel_10000_EE2_LCDM-randoms_kmin-0.0_kmax-2.5__log.txt 
The following line has not been formatted: 
 group_lab_dict {'True': 'non_lcdm', 'dgp': 'non_lcdm', 'ds'

## Load modules for classification of a single spectrum

### Functions to load trained models and data
just execute the following cells

In [ ]:
def read_pre_trained_flags(args_dict):
  '''
  Read model options from logfile and return them in the form of FLAGS (the corresponding class is in utils)
  '''
  args = DummyFlags(args_dict)
  FLAGS = get_flags(args.log_path)
  if args.models_dir is not None:
        print('Reading model from the directory %s' %args.models_dir)
        FLAGS.models_dir = args.models_dir
  return FLAGS


def get_pre_trained_model(fname, n_classes, input_shape):
  '''
  returns pre-trained model.
  Input:  - fname: string, folder where the trained model is stored
          - n_classes: integer, number of output classes of the model
          - input_shape:  tuple, input dimensions of the model.
                          For pre-trained model in the paper use (100, 4)

  '''

  log_path = os.path.join('models', fname, fname+'_log.txt' )
  models_dir = os.path.join('models', fname)

  args_dict= {'log_path':log_path, 'models_dir': models_dir, }
  FLAGS=read_pre_trained_flags(args_dict)

  print('Input shape %s' %str(input_shape))


  model_loaded =  load_model_for_test(FLAGS, input_shape, n_classes=n_classes,
                                        generator=None, new_fname='')


  return model_loaded



def load_X(fname, sample_pace, i_max, norm_data,
           add_noise=True, normalise=True, use_sample_pace=False, include_k=True):
  '''
  Generates one power spectrum X ready for classification.
  Options:  fname: path to data
            sample_pace: read one point every sample_pace. Useful to reduce data dimension
            i_max: Index corresponding to the max k used
            norm_data: normalization spectrum

            add_noise: whether to add noise or not
            normalise: normalise by Planck or not
            use_sample_pace
            include_k: return k, X or only X



  '''

  all = np.loadtxt(fname)
  print('loaded data shape: %s' %str(all.shape))
  if include_k:
    k, X = all[:, 0], all[:, 1:]
    print('k, X  shape: %s, %s' %(str(k.shape), str(X.shape)))
  else:
    X = all
    k=np.arange(X.shape[0])
    print('X  shape : %s' %( str(X.shape)))
  if use_sample_pace:
    k, X = k[::sample_pace], X[::sample_pace]
  print('X  shape after sample pace:  %s' %( str(X.shape)))
  k, X  = k[:i_max], X[:i_max]
  print('X  shape after k max: %s' %( str(X.shape)))
  if add_noise:
    noise = np.random.normal(loc=0, scale=generate_noise(k,X, add_sys=True,add_shot=True,sigma_sys=5 ))
    X = X+noise
  if normalise:
    planck = norm_data
    print('planck data shape: %s' %str(planck.shape))
    X = X/planck-1
  if include_k:
    return k, X
  else:
    return X

## Code for getting confidence

In [ ]:
def sample_from_data(model,  my_Xs_norm,  my_min=np.zeros(5), my_max=np.ones(5),
                     n_samples=50000, num_monte_carlo=100, th_prob=0.5,
                     verbose=True):
  '''
  Given some normalized features my_Xs_norm and the model, compute mu and Sigma as in the paper eq. 2.8-2.9
  and return samples from the probability distribuion eq. 2.10
  '''
  Al_error, Ep_error, mean = get_mu_Sigma(model, my_Xs_norm , num_monte_carlo=num_monte_carlo, th_prob=th_prob, verbose=verbose)
  Sigma = (Al_error+ Ep_error).numpy()
  MM = mean.numpy()
  if verbose:
    print('Sigma')
    print(np.round(Sigma, 3))
  samples, z_samples = sample_probas(Sigma, MM, my_min=my_min,my_max=my_max, n_samples=n_samples, verbose=verbose)

  return samples, z_samples, Sigma, MM



def get_mu_Sigma(model, my_Xs_norm, num_monte_carlo=100, th_prob=0.5, verbose=True):
  ''''
  Given some normalized features my_Xs_norm and the model, compute mu and Sigma as in the paper eq. 2.8-2.9
  '''
  if verbose:
    print('Computing mu and sigma...')
  sampled_probas, _, _ = my_predict(tf.expand_dims(my_Xs_norm, axis=0), model, num_monte_carlo=num_monte_carlo, th_prob=th_prob)
  mean = tf.reduce_mean(sampled_probas[:,0,:], axis=0)
  _, Al_error, Ep_error = get_err_variances(sampled_probas[:,0,:])
  return Al_error, Ep_error, mean


def sample_probas(Sigma, MM, my_min=np.zeros(5), my_max=np.ones(5), n_samples=50000, verbose=True):
  '''
  Given mu and Sigma as in the paper eq. 2.8-2.9, computes B, U as described in appendix B of the paper.
  Then calls function to sample the prob. distribution (eq. 2.10)
  '''
  if verbose:
    print('Sample probas call')
  evals, evecs = np.linalg.eig(Sigma)
  B = evecs
  U = B.T @ Sigma @  B
  if verbose:
    print('U')
    print(np.round(U, 3))
  n_dims=U.shape[0]-1
  n_dims=U.shape[0]-1
  ind_min = np.argmin(np.diag(U))
  if verbose:
    print('Index of min eigenvalue is: %s' %ind_min)
    print('Min eigenvalue is: %s' %np.min(np.diag(U)))
  UU = np.array([np.sqrt(U[i,i])  for i in range(U.shape[0]) if i!=ind_min ] )
  if verbose:
    print('U has %s elements ' %str(UU.shape[0]))
  samples, z_samples = get_samples(UU, B, MM, my_min=my_min, my_max=my_max, ind_min=ind_min, n_dims=n_dims, n_samples=n_samples, verbose=verbose)
  return samples, z_samples


def get_samples(UU, B, MM, my_min=np.zeros(5), my_max=np.ones(5), ind_min=-1, n_dims=4, n_samples=50000, verbose=True):
  '''
  samples eq. 2.10 as described in appendix B
  '''
  if verbose:
    print('Sampling...')
  samples = np.zeros((0, n_dims+1))
  z_samples = np.zeros((0, n_dims))
  while samples.shape[0] < n_samples:
    s = np.random.multivariate_normal( np.zeros(n_dims), np.diag(UU**2), size=(n_samples,))
    accepted = s[(np.min(X_val_batch(s, B, MM, null_ind=ind_min)-my_min, axis=1) >= 0) & (np.max(X_val_batch(s, B, MM, null_ind=ind_min) - my_max, axis=1) <= 0)]
    samples = np.concatenate((samples, X_val_batch(accepted, B, MM, null_ind=ind_min)), axis=0)
    z_samples = np.concatenate((z_samples, accepted), axis=0)
  samples = samples[:n_samples, :]
  z_samples = z_samples[:n_samples, :]
  if verbose:
    print('Done.')
  return samples, z_samples



def X_val_batch(Z, B, MM, null_ind):
  return np.array([ X_val(Z[i], B, MM, null_ind=null_ind,) for i in range(Z.shape[0])])

def Z(X, B, MM):
  '''  X must be 5-d '''
  return B.T @ (X-MM)

def X_val(Z, B, MM, null_ind=-1, verbose=False):
  '''  Z must be 4-d , MM must be 5-d '''
  if verbose:
    print(Z)
  Z = np.insert(Z, null_ind, 0)
  if verbose:
    print(Z)
  return B @ Z+ MM


def get_err_variances(prob_k, AlEp_split=True):
  '''
  Computes aleatoric and epistemic uncertainty from MC samples from the net. weights
  '''
  num_samples = prob_k.shape[0]
  prob_mean = tf.reduce_mean(prob_k, axis=0)

  diag_probs = tf.stack([np.diag(prob_k[i,:]) for i in range(num_samples)], axis=0)  # Diagonal matrix whose diagonal elements are the output of the network for the ith sample
  outer_products_Al = tf.stack([tf.tensordot(prob_k[i,:], prob_k[i,:], axes =0) for i in range(num_samples)], axis=0) # Outer self-product of the predictive vectors for the ith sample used in computing Al uncertainty
  outer_products_Ep = tf.stack([tf.tensordot(prob_k[i,:] - prob_mean, prob_k[i,:] - prob_mean, axes =0) for i in range(num_samples)], axis=0) # Outer self-product between the difference of the predictive vector for ith sampl and average prediction

  Al_error = tf.reduce_mean(diag_probs - outer_products_Al, axis=0) # Average over all the samples to compute aleotoric uncertainty
  Ep_error = tf.reduce_mean(outer_products_Ep, axis=0) # Average over all the samples to compute epistemic uncertainty

  tot_error = Al_error + Ep_error # Combine to compute total uncertainty
  confidences = tf.linalg.diag_part(tot_error).numpy()

  if AlEp_split:
    return confidences, Al_error, Ep_error
  else:
    return confidences



def get_P_from_samples(samples, th_prob=0.5 ):
  '''
  Given samples from eq. 2.10, computes the probabilities as in eq. 2.11
  '''
  Ntot = samples.shape[0] #-unclassified
  n_dims  = samples.shape[-1]
  all_preds = np.array([predict_bayes_label(sample, th_prob=th_prob) for sample in samples])
  unclassified=all_preds[all_preds==99].shape[0]

  n_in_class = np.array([all_preds[all_preds==k].shape[0] for k in range(n_dims)])

  #print('Ntot: %s  ' %Ntot)
  p_uncl=unclassified/Ntot
  #print('%s unclassified examples ' %unclassified)
  #print('P(unclassified)=%s ' %p_uncl)
  pp=n_in_class/Ntot
  #print(pp)
  #print(p_uncl)

  return np.append(pp, p_uncl)


def get_all_probas(X, model_loaded,
                   n_classes=5,
                   num_monte_carlo=100,
                   th_prob=0.5,
                   n_samples=10000,
                   verbose=True):
    '''
    Given features X, predicts the labels with MC samples from the net,
    computes mu and sigma, samples from the prob distribution
    and computes P.
    Returns result in the form of a dictionary
    '''

    # Gaussian approx
    samples, z_samples, Sigma, MM = sample_from_data(model_loaded,
                                                 my_Xs_norm=X,
                                                 my_min=np.zeros(n_classes), my_max=np.ones(n_classes),
                                                 n_samples=n_samples,
                                                 num_monte_carlo=num_monte_carlo,
                                                 th_prob=th_prob, verbose=verbose)

    P = get_P_from_samples(samples, th_prob=th_prob)
    if verbose:
      print('P: %s ' %str(P))
      print('P sum to %s' %P.sum())
    if verbose:
      print('mu  sum to %s' %MM.sum())


    # Result
    res = {'samples':samples, 'P':P, 'z_samples': z_samples, 'Sigma': Sigma, 'MM': MM,}
    return res


def plot_hist_1D(samples, MM, th_prob=0.5,
                  P=None, inv_labels_dict=None,):
  '''
  Plots histogram of gauss samples for each class
  '''

  n_dims = samples.shape[-1]-1
  fig, axs = plt.subplots(1, n_dims+1, sharey=True, sharex=False,figsize=(25,5))

  t_str=''
  for k in range(n_dims+1):

    axs[k].hist(samples.T[k], bins=15, color = "lightgray",lw=0, density=True, label='Gauss samples')
    axs[k].text(0.1, 0.9, '$\mu = $' + '$'+ str(MM[k])+'$', fontsize = 16 , transform=axs[k].transAxes);
    if P is  not None:
      axs[k].text(0.1, 0.8, '$ P = $' + '$'+ str(P[k])+'$', fontsize = 16 , transform=axs[k].transAxes);
      if k==0:
        t_str+= 'P unclassified: %s \n' %str(np.round(P[-1], 2) )
    axs[k].set_xlim(0,1)
    axs[k].set_ylim(0,10)

    if inv_labels_dict is not None:
      axs[k].set_title(inv_labels_dict[k])


    axs[k].legend(loc='lower right')

  axs[0].set_ylabel('P')
  fig.suptitle(t_str)

  plt.show()

## Load models

Here we load the pre trained models

In [ ]:
# Encoding of the labels for the two networks. Can be found in the log file of the training.
# Here it is easier to define it by hand.

inv_labels_dict_6={'dgp': 0, 'ds': 1, 'fr': 2, 'lcdm': 3, 'rand': 4, 'wcdm': 5}
inv_labels_dict_2={0:'lcdm', 1:'non-LCDM'}

change the model name (first parameter) in the following function

In [ ]:
model_6 = get_pre_trained_model('ds_ee2_1k_equalweight_randoms_kmax_5', 6, (100, 4) )

## Loading a trained network and applying it to a single file

Template for loading spectra. Suppose you saved the spectrum in
'data/example_spectra/my_ex.txt' . The spectrum is in 4 redshift bins and with 100 points between 0.01 - 2.5 in k . We normalize by the reference planck spectrum and add gaussian noise, then pass it to the five-label network.

Load normalization data first

In [ ]:
sample_pace=4 # the planck data were generated with 500 points up to k=10 . We need 1 every 4 points, and to cut to k_max=5.0
i_max=200 #for kmax = 5

planck = np.loadtxt('data/normalisation/planck_ee2.txt')[:, 1:][::sample_pace][:i_max]

In [ ]:
k, X_ds = load_X('data/ds_ee2_test_1k/ds/976.txt',
                 sample_pace=sample_pace, i_max=200,
                 norm_data=planck,
                 add_noise=False, normalise=True, use_sample_pace=True)

Let's look at the input features as seen by the network.

In [ ]:
plt.plot(k, X_ds);
plt.xscale('log');

## Classify your spectra and compute confidence
### (for Bayesian networks only)

In [ ]:
res_ds = get_all_probas(X_ds,
                        model_5,
                        n_classes=6,
                        num_monte_carlo=10,
                        th_prob=0.5,
                        n_samples=1000)

In [ ]:
print('P-non-LCDM=%s' %(np.delete(res_ds['P'], [2]).sum()))
plot_hist_1D(res_ds['samples'],
             res_ds['MM'],
             th_prob=0.5,
             P=res_ds['P'],
             inv_labels_dict=inv_labels_dict_5)
